# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "pycityschoolccsv/schools_complete.csv"
student_data_to_load = "pycityschoolccsv/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#student data to be used later in the program
Student_Data = pd.DataFrame(student_data)
Math_Data = Student_Data[['school_name', 'grade', 'math_score' ]]
Reading_Data = Student_Data[['school_name', 'grade', 'reading_score']]

# combining data into a single dataform
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
Combined_Data = pd.DataFrame(school_data_complete)
Combined_Data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [16]:
#calculating summary matrics for the districts (all schools combined)
#Calculate Total Passing Math
Mathabove70 = Combined_Data['math_score'] >= 70
HowManyAbove70 = Combined_Data[Mathabove70]
MathStudentsAbove70 = HowManyAbove70['math_score'].count()

#Calculate Total Passing Reading
Readingabove70 = Combined_Data['reading_score'] >=70
TotalReadingAbove70=Combined_Data[Readingabove70]
ReadingStudentsAbove70 = TotalReadingAbove70['reading_score'].count()

#Calculate total number of schools
School_Total = Combined_Data.school_name.nunique()
#Calculate Total Number of Students
Student_Total = Combined_Data.student_name.count()
#Calculate Total Budget for all schools
Budget_Total = school_data.budget.sum()
#Calculate Avg. Math Score
Math_Avg = Combined_Data.math_score.mean()
#Calculate Avg. Reading Score
Reading_Avg = Combined_Data.reading_score.mean()
#Percentage of math above 70
Math_Passing = (MathStudentsAbove70 / Student_Total)
#Percentage of Reading above 70
Reading_Passing = (ReadingStudentsAbove70 / Student_Total)
#Percentage of Overall Passing Students
Overall_Passing = (Math_Passing + Reading_Passing)/2

####Create DataFrame####
District_Summary = {'Total Schools':(School_Total),
                     'Total Students': '{:,.0f}'.format(Student_Total),
                   'Total Budget':'${:,.2f}'.format(Budget_Total),
                    'Average Math Score':(Math_Avg),
                    'Average Reading Score':(Reading_Avg),
                    '% Passing Math':'{:,.6f}'.format((Math_Passing)*100),
                    '% Passing Reading':'{:,.6f}'.format((Reading_Passing)*100),
                    '% Overall Passing Rate':'{:,.6f}'.format((Overall_Passing)*100)
                   }
District_Summary_DF = pd.DataFrame(District_Summary, index=[0])
District_Summary_DF


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [17]:
# Creating an overview Dataframe table summerizing each of the above indices for each school

# Math score % per school
Reduced_MathDB = Combined_Data.loc[Combined_Data['math_score']>=70]
Math_DB = Reduced_MathDB.groupby('school_name').agg({'math_score':'count'})

# Reading score % per school
Reduced_ReadingDB = Combined_Data.loc[Combined_Data['reading_score']>=70]
Reading_DB = Reduced_ReadingDB.groupby('school_name').agg({'reading_score':'count'})

# Student Size for school
Size_DB = Combined_Data.groupby('school_name').agg({'size':'mean'})

# Combine Math, Reading, and Size - merge by 
Stem_DB = pd.merge(pd.merge(Math_DB, Reading_DB, on='school_name'), Size_DB, on='school_name')

# calculating % passing rates for reading, math and overall for each schoool

ReadingPerc = Stem_DB['reading_score'] / Stem_DB['size']
MathPerc = Stem_DB['math_score'] / Stem_DB['size']
Stem_DB['Math Passing'] = MathPerc*100
Stem_DB['Reading Passing'] = ReadingPerc*100
Stem_DB['Overall Passing'] = (Stem_DB['Math Passing'] + Stem_DB['Reading Passing']) / 2
#Stem_DB
Stem_DB = Stem_DB[['size', 'Math Passing', 'Reading Passing', 'Overall Passing']]

# computing total budget, and average % scores for math and reading for each school 
School_DF = Combined_Data[['school_name', 'budget', 'math_score','reading_score',]]
Grouped_DF = School_DF.groupby('school_name').agg({
    'budget':'mean',
    'math_score':'mean',
    'reading_score':'mean'})

# Merging 
School_SummaryPREDF = pd.merge(Grouped_DF, Stem_DB, on='school_name')

# computing cost per student for each school
AverageCost = School_SummaryPREDF['budget']/School_SummaryPREDF['size']
School_SummaryPREDF["Cost_Per"] = AverageCost



# Identify schools by school type and merging the metrics

SchoolType = school_data[['school_name','type']]
School_SummaryDF = pd.merge(School_SummaryPREDF, SchoolType, on='school_name')

#Re-Ordering the Columns in School SummaryDF
School_SummaryDF = School_SummaryDF[[
    'school_name','type', 'size', 'budget', 'Cost_Per', 'math_score', 'reading_score',
    'Math Passing', 'Reading Passing', 'Overall Passing']]
School_SummaryDF

# Renaming and formating column names
School_SummaryDF = School_SummaryDF.rename(columns={
    "school_name":"School\nName",
    "type":"School\nType",
    "size":"Total\nStudents",
    "budget":"Total School\nBudget",
    "Cost_Per":"Per Student\nBudget",
    "math_score":"Average Math\nScore",
    "reading_score":"Average Reading\nScore",
    "Math Passing":"% Passing\nMath",
    "Reading Passing":"% Passing\nReading",
    "Overall Passing":"% Overall Passing\nRate"
    })
School_SummaryDF["Total School\nBudget"] = School_SummaryDF["Total School\nBudget"].map("${:,.2f}".format)
School_SummaryDF["Per Student\nBudget"] = School_SummaryDF["Per Student\nBudget"].map("${:.2f}".format)

School_SummaryDF.head(3)




,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [18]:
# Sorting the five top performing schools in the district by the passing rate

TopPerformingSchools = School_SummaryDF.sort_values('% Overall Passing\nRate', ascending=False)

TopPerformingSchools = TopPerformingSchools.reset_index(drop=True)
TopPerformingSchools = TopPerformingSchools.set_index("School\nName")
TopPerformingSchools.head(5)




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [19]:
# List of the five schools with wurst performance in the district by passing rate.

BottomPerformingSchools = School_SummaryDF.sort_values('% Overall Passing\nRate', ascending=True)

BottomPerformingSchools = BottomPerformingSchools.reset_index(drop=True)
BottomPerformingSchools = BottomPerformingSchools.set_index("School\nName")
BottomPerformingSchools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [20]:
# Math score by grade (9th, 10th, 11th and 12th) for each school in the distict

NinthGrade = Math_Data.loc[Math_Data['grade']=='9th']
NinthGradeGrouped = NinthGrade.groupby('school_name').agg({'math_score':'mean'})
NinthGradeGrouped = NinthGradeGrouped.rename(columns={"math_score":"9th"})
TenthGrade = Math_Data.loc[Math_Data['grade']=='10th']
TenthGradeGrouped = TenthGrade.groupby('school_name').agg({'math_score':'mean'})
TenthGradeGrouped = TenthGradeGrouped.rename(columns={"math_score":"10th"})
EleventhGrade = Math_Data.loc[Math_Data['grade']=='11th']
EleventhGradeGrouped = EleventhGrade.groupby('school_name').agg({'math_score':'mean'})
EleventhGradeGrouped = EleventhGradeGrouped.rename(columns={'math_score':'11th'})
TwelfthGrade = Math_Data.loc[Math_Data['grade']=='12th']
TwelfthGradeGrouped = TwelfthGrade.groupby('school_name').agg({'math_score':'mean'})
TwelfthGradeGrouped = TwelfthGradeGrouped.rename(columns={'math_score':'12th'})
GroupedMath = pd.merge(pd.merge(pd.merge(NinthGradeGrouped, TenthGradeGrouped, on='school_name'),
                                EleventhGradeGrouped, on='school_name'),
                                TwelfthGradeGrouped, on='school_name')


Student_Data = pd.DataFrame(student_data)
Math_Data = Student_Data[['school_name', 'grade', 'math_score' ]]
Reading_Data = Student_Data[['school_name', 'grade', 'reading_score']]

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
Combined_Data = pd.DataFrame(school_data_complete)

Combined_Data.head()
GroupedMath


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [21]:
# Reading score by grade (9th, 10th, 11th and 12th) for each school in the distict

#9th Grade Reading Scores 
NinthGradeR = Reading_Data.loc[Reading_Data['grade']=='9th']
NinthGradeGroupedR = NinthGradeR.groupby('school_name').agg({'reading_score':'mean'})
NinthGradeGroupedR = NinthGradeGroupedR.rename(columns={'reading_score':"9th"})

#10th Grade Reading Scores
TenthGradeR = Reading_Data.loc[Reading_Data['grade']=='10th']
TenthGradeGroupedR = TenthGradeR.groupby('school_name').agg({'reading_score':'mean'})
TenthGradeGroupedR = TenthGradeGroupedR.rename(columns={'reading_score':"10th"})

#11th Grade Reading Scores
EleventhGradeR = Reading_Data.loc[Reading_Data['grade']=='11th']
EleventhGradeGroupedR = EleventhGradeR.groupby('school_name').agg({'reading_score':'mean'})
EleventhGradeGroupedR = EleventhGradeGroupedR.rename(columns={'reading_score':'11th'})

#12th Grade Reading Scores
TwelfthGradeR = Reading_Data.loc[Reading_Data['grade']=='12th']
TwelfthGradeGroupedR = TwelfthGradeR.groupby('school_name').agg({'reading_score':'mean'})
TwelfthGradeGroupedR = TwelfthGradeGroupedR.rename(columns={'reading_score':'12th'})
GroupedReading = pd.merge(pd.merge(pd.merge(NinthGradeGroupedR, TenthGradeGroupedR, on='school_name'),
                                EleventhGradeGroupedR, on='school_name'),
                                TwelfthGradeGroupedR, on='school_name')
GroupedReading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [24]:
#school performance scores based on spending
# Spending bins.
bins = [0, 500, 600, 630, 680]
bin_name = ["<$500", "$501-600", "$601-630", "$631-680"]

#keeping unformated values as floats or ints 
School_SpendingDF = School_SummaryPREDF[['Cost_Per', 'math_score', 'reading_score', 'Math Passing',
       'Reading Passing', 'Overall Passing']]

#Renaming the columns
School_SpendingDF = School_SpendingDF.rename(columns={
    "math_score":"Average Math\nScore",
    "reading_score":"Average Reading\nScore",
    "Math Passing":"% Passing\nMath",
    "Reading Passing":"% Passing\nReading",
    "Overall Passing":"% Overall Passing\nRate"})

#Inserting bin columns
School_SpendingDF["Spending Ranges (Per Student)"] = pd.cut(School_SpendingDF["Cost_Per"], bins, labels=bin_name)

#grouping bin Column
Scores_School_Spending = School_SpendingDF.groupby('Spending Ranges (Per Student)')

#renaming the Columns
Scores_School_Spending_Bin = Scores_School_Spending[["Average Reading\nScore","Average Reading\nScore","% Passing\nMath",
                                                     "% Passing\nReading","% Overall Passing\nRate"]]
Scores_School_Spending_Bin.max()

,Average Reading Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$500,NaN,NaN,NaN,NaN,NaN
$501-600,83.989488,83.989488,94.133477,97.039828,95.586652
$601-630,84.044699,84.044699,94.594595,97.138965,95.270270
$631-680,83.848930,83.848930,93.272171,97.308869,95.290520


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
#school performance scores according to size
# creating bins and bin group namess.
size_bins = [0, 1000, 2000, 5000]
size_groupnames = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Reference unformated values so they are floats or ints - not formatted objects
School_SizeDF = School_SummaryPREDF[['size', 'math_score', 'reading_score', 'Math Passing',
       'Reading Passing', 'Overall Passing']]

#renameing the columns
School_SizeDF = School_SizeDF.rename(columns={
    "math_score":"Average Math\nScore",
    "reading_score":"Average Reading\nScore",
    "Math Passing":"% Passing\nMath",
    "Reading Passing":"% Passing\nReading",
    "Overall Passing":"% Overall Passing\nRate"})

#inserting bin columns
School_SizeDF["School Size"] = pd.cut(School_SizeDF["size"], size_bins, labels=size_groupnames)

#Groupby Bin Column
Scores_School_Size = School_SizeDF.groupby('School Size')

#renaming the columns
Scores_School_Size_Bin = Scores_School_Size[["Average Reading\nScore","Average Reading\nScore","% Passing\nMath",
                                                     "% Passing\nReading","% Overall Passing\nRate"]]
Scores_School_Size_Bin.max()

,Average Reading Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),84.044699,84.044699,94.594595,96.252927,95.270270
Medium (1000-2000),83.975780,83.975780,94.133477,97.308869,95.586652
Large (2000-5000),83.989488,83.989488,93.867718,96.539641,95.203679


## Scores by School Type

* Perform the same operations as above, based on school type.

In [26]:
# Scored based on the school type

# Grouping based on the school type
ScoresByType = School_SummaryDF.groupby("School\nType")

#grouping columns with the soring indices 
ScoresBySchoolType = ScoresByType.agg({
    "Average Math\nScore":'mean',
    "Average Reading\nScore":'mean',
    "% Passing\nMath":'mean',
    "% Passing\nReading":'mean',
    "% Overall Passing\nRate":'mean'})

#Printing table
ScoresBySchoolType

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [ ]:
# THIS WAS ONE LONG AND CHALLENGING EXERCISE WHICH TOOK ME FOREVER TO COMPLETE YET FORCED ME TO LEARN A LOTTTT. 
